# Example of re-expansion of the H₂CO PES using identical internal coordinate system

In [1]:
import jax
import jax.numpy as jnp
import numpy as np

from vibrojet.basis_utils import generate_prod_ind
from vibrojet.potentials import h2co_AYTY
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Coordinate mapping

In [2]:
def internal_to_morse(q):
    r0, r1, r2, a1, a2, cos_tau = q
    r0eq, r1eq, r2eq, a1eq, a2eq, _ = h2co_AYTY.mep(cos_tau)
    y1 = 1 - jnp.exp(-(r0 - r0eq))
    y2 = 1 - jnp.exp(-(r1 - r1eq))
    y3 = 1 - jnp.exp(-(r2 - r2eq))
    y4 = a1 - a1eq
    y5 = a2 - a2eq
    y6 = 1 + cos_tau
    return jnp.array([y1, y2, y3, y4, y5, y6])


def morse_to_internal(y):
    y1, y2, y3, y4, y5, y6 = y
    cos_tau = y6 - 1
    r0eq, r1eq, r2eq, a1eq, a2eq, _ = h2co_AYTY.mep(cos_tau)
    r0 = -jnp.log(1 - y1) + r0eq
    r1 = -jnp.log(1 - y2) + r1eq
    r2 = -jnp.log(1 - y3) + r2eq
    a1 = y4 + a1eq
    a2 = y5 + a2eq
    return jnp.array([r0, r1, r2, a1, a2, cos_tau])

Generate expansion for PES

In [3]:
ncoo = 6

max_pow = 6
powers = [np.arange(max_pow + 1)] * ncoo
deriv_ind, deriv_mind = next(
    generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_pow)
)

print("max expansion power:", max_pow)
print("number of expansion terms:", len(deriv_ind))

tau0 = np.pi  # equilibrium value of the `book` dihedral angle
cos_tau0 = np.cos(tau0)

q0 = h2co_AYTY.mep(cos_tau0)
y0 = internal_to_morse(q0)

print("equilibrium values of r-alpha-costau coordinates:", q0)
print("equilibrium values of morse-dalpha-dcostau coordinates:", y0)

poten_coefs = deriv_list(
    lambda x: h2co_AYTY.poten(morse_to_internal(x)), deriv_ind, y0, if_taylor=True
)

max expansion power: 6
number of expansion terms: 924
equilibrium values of r-alpha-costau coordinates: [ 1.2   1.1   1.1   2.13  2.13 -1.  ]
equilibrium values of morse-dalpha-dcostau coordinates: [0. 0. 0. 0. 0. 0.]
Time for d= 0 : 11.3 s
Time for d= 1 : 11.42 s
Time for d= 2 : 22.14 s
Time for d= 3 : 36.36 s
Time for d= 4 : 56.01 s
Time for d= 5 : 79.55 s
Time for d= 6 : 111.88 s


Print computed coefficients and compare with the original PES expansion coefficients

In [8]:
thresh = 1e-6
for i in range(len(deriv_ind)):
    if np.abs(poten_coefs[i] > thresh):
        ind = np.where((h2co_AYTY.ind == deriv_ind[i]).all(axis=1))[0][0]
        coef = h2co_AYTY.coefs[ind]
        print(
            deriv_ind[i],
            "%20.6f" % poten_coefs[i],
            "%20.6f" % coef,
            "%20.6f" % (poten_coefs[i] - coef),
        )

[0 0 0 0 0 1]         13239.727881         13239.727881             0.000000
[0 0 0 0 0 2]          4627.962169          4627.962169             0.000000
[0 0 0 0 0 3]          1439.478742          1439.478742             0.000000
[0 0 0 0 0 4]          1006.758455          1006.758455             0.000000
[0 0 0 0 2 0]         31402.651686         31402.651686             0.000000
[0 0 0 0 4 0]           463.426455           463.426455             0.000000
[0 0 0 1 1 0]         21264.254862         21264.254862             0.000000
[0 0 0 1 1 2]          8603.647823          8603.647823             0.000000
[0 0 0 1 2 0]         15666.762364         15666.762364             0.000000
[0 0 0 1 2 1]          6946.931759          6946.931759             0.000000
[0 0 0 1 3 0]          3284.199296          3284.199296             0.000000
[0 0 0 1 3 1]          1844.137939          1844.137939             0.000000
[0 0 0 1 4 0]          2960.387858          2960.387858             0.000000